# Microsoft Graph Meeting Insights API Interaction (Delegated)

## 1. Setup and Prerequisites

This notebook demonstrates how to call the Microsoft Graph API to retrieve insights from a Teams meeting transcript using **delegated user permissions**.

### API Permissions Required
Your Azure App Registration will need the following **delegated** permissions for this notebook to function correctly:
- `User.Read`
- `Calendars.Read`
- `OnlineMeetings.Read`
- `OnlineMeetingTranscript.Read.All`
- `OnlineMeetingAiInsight.Read.All`

> **Note**: The `OnlineMeetingTranscript.Read.All` and `OnlineMeetingAiInsight.Read.All` permissions require administrator consent before they can be granted.

### Enable Public Client Flow
For delegated authentication to work in this script, you must enable the public client flow in your App Registration:
1. In your App Registration, go to the **Authentication** tab.
2. Scroll down to **Advanced settings**.
3. Set the toggle for **Allow public client flows** to **Yes**.
4. Click **Save**.
5. Ensure you also have a **Mobile and desktop applications** platform configured with a redirect URI for `http://localhost`.

### Environment Variables
This notebook uses a `.env` file in the root of the repository to load configuration. Make sure you have copied `.env-example` to `.env` and filled in the required values, especially:
- `CopilotAPIPythonClient_Tenant`
- `CopilotAPIPythonClient_Id`

In [ ]:
!pip install msal requests python-dotenv

## 2. Authentication

First, we'll authenticate with Microsoft Graph using the MSAL library. We will acquire an access token that will be used for all subsequent API calls.

In [ ]:
import os
import msal
import requests
import json
from dotenv import load_dotenv
import base64
from urllib.parse import quote

# Load environment variables from .env file
# Assumes the .env file is in the root of the repository
load_dotenv(dotenv_path='../.env')

TENANT_ID = os.getenv('CopilotAPIPythonClient_Tenant')
CLIENT_ID = os.getenv('CopilotAPIPythonClient_Id')

authority = f"https://login.microsoftonline.com/{TENANT_ID}"
scopes = [
    "https://graph.microsoft.com/User.Read",
    "https://graph.microsoft.com/Calendars.Read",
    "https://graph.microsoft.com/OnlineMeetings.Read",
    "https://graph.microsoft.com/OnlineMeetingTranscript.Read.All",
    "https://graph.microsoft.com/OnlineMeetingAiInsight.Read.All"
]

app = msal.PublicClientApplication(
    CLIENT_ID,
    authority=authority,
    token_cache=msal.SerializableTokenCache() # Persist token cache
)

result = None
accounts = app.get_accounts()
if accounts:
    print("Found account(s) in cache. Attempting to acquire token silently.")
    result = app.acquire_token_silent(scopes, account=accounts[0])

if not result:
    print("No suitable token in cache. Let's get a new one interactively.")
    result = app.acquire_token_interactive(scopes=scopes)

if "access_token" in result:
    access_token = result['access_token']
    print("Access Token acquired successfully.")
else:
    print(result.get("error"))
    print(result.get("error_description"))
    print(result.get("correlation_id"))
    access_token = None

## 3. Get User Info and Calendar Events

Now we'll use the access token to call the Graph API. First, we get the logged-in user's information. Then, we'll retrieve their upcoming calendar events to find a meeting `joinWebUrl`.

In [ ]:
from datetime import datetime, timedelta, timezone
from urllib.parse import quote

headers = {'Authorization': f'Bearer {access_token}'}

# Get User Info from /me endpoint
user_endpoint = "https://graph.microsoft.com/v1.0/me"
user_response = requests.get(user_endpoint, headers=headers)
user_data = user_response.json()
user_upn = user_data.get('userPrincipalName')
print(f"Operating as user: {user_upn}\n")

# Define the date range for calendar events
# Look BACKWARDS for past meetings that could have transcripts
end_date = datetime.now(timezone.utc).strftime('%Y-%m-%dT%H:%M:%S') + 'Z'
start_date = (datetime.now(timezone.utc) - timedelta(days=30)).strftime('%Y-%m-%dT%H:%M:%S') + 'Z'

print(f"Looking for meetings between {start_date} and {end_date}\n")

# Get Calendar Events for the logged-in user within the date range
# Use calendarView for time-based queries
events_endpoint = (
    f"https://graph.microsoft.com/v1.0/me/calendarView"
    f"?startDateTime={start_date}&endDateTime={end_date}"
)
events_response = requests.get(events_endpoint, headers=headers)
events_data = events_response.json()

meetings = [] # Clear the list before populating
if 'value' in events_data:
    for event in events_data['value']:
        online_meeting = event.get('onlineMeeting')
        if online_meeting and online_meeting.get('joinUrl'):
            meetings.append({
                'subject': event['subject'],
                'joinUrl': online_meeting['joinUrl']
            })

print(f"Found {len(meetings)} online meeting(s):")
for i, meeting in enumerate(meetings):
    print(f"{i}: {meeting['subject']}")

# Prompt user to select a meeting
meeting_choice = int(input("\nEnter the number of the meeting to analyze: "))
selected_meeting_url = meetings[meeting_choice]['joinUrl']
print(f"Selected: {meetings[meeting_choice]['subject']}")
print(f"Join URL: {selected_meeting_url}\n")

## 4. Get Online Meeting and Transcript ID

Select a meeting from the list above and enter its number. We will use the `joinWebUrl` to find the `onlineMeeting` resource ID. This ID is required to find the associated transcript.

In [ ]:
# The joinWebUrl needs to be encoded for the filter parameter
encoded_url = base64.b64encode(selected_meeting_url.encode()).decode()
print (encoded_url)

online_meeting_endpoint = f"https://graph.microsoft.com/v1.0/me/onlineMeetings?$filter=JoinWebUrl eq '{selected_meeting_url}'"
online_meeting_response = requests.get(online_meeting_endpoint, headers=headers)
online_meeting_data = online_meeting_response.json()

meeting_id = None
if 'value' in online_meeting_data and len(online_meeting_data['value']) > 0:
    meeting_id = online_meeting_data['value'][0]['id']
    print(f"Found Meeting ID: {meeting_id}")
else:
    print("Could not find an online meeting with that Join URL.")

transcript_id = None
if meeting_id:
    transcripts_endpoint = f"https://graph.microsoft.com/v1.0/me/onlineMeetings/{meeting_id}/transcripts"
    transcripts_response = requests.get(transcripts_endpoint, headers=headers)
    transcripts_data = transcripts_response.json()
    if 'value' in transcripts_data and len(transcripts_data['value']) > 0:
        transcript_id = transcripts_data['value'][0]['id']
        print(f"Found Transcript ID: {transcript_id}")
    else:
        print("No transcripts found for this meeting.")

## 5. Get Transcript Content

With the meeting and transcript IDs, we can now fetch the transcript content. We'll request it in VTT format.

In [ ]:
if transcript_id:
    transcript_content_endpoint = f"https://graph.microsoft.com/v1.0/me/onlineMeetings/{meeting_id}/transcripts/{transcript_id}/content?$format=text/vtt"
    transcript_content_response = requests.get(transcript_content_endpoint, headers=headers)
    print(transcript_content_endpoint)
    if transcript_content_response.status_code == 200:
        print("--- Transcript Content ---")
        print(transcript_content_response.text)
    else:
        print(f"Error fetching transcript content: {transcript_content_response.status_code}")
        print(transcript_content_response.text)

## 6. Get Meeting Insights

Finally, we call the `insights` endpoint for the meeting to get the structured insights data.

In [ ]:
if meeting_id:
    # Note: The insights API is only available in the beta endpoint
    user_id = user_data.get('id')
    insights_list_endpoint = f"https://graph.microsoft.com/beta/copilot/users/{user_id}/onlineMeetings/{meeting_id}/aiInsights"
    insights_list_response = requests.get(insights_list_endpoint, headers=headers)
    
    if insights_list_response.status_code == 200:
        insights_list_data = insights_list_response.json()
        
        if 'value' in insights_list_data and len(insights_list_data['value']) > 0:
            print(f"Found {len(insights_list_data['value'])} AI insight(s)\n")
            
            for insight in insights_list_data['value']:
                insight_id = insight.get('id')
                print(f"--- Fetching Insight ID: {insight_id} ---")
                
                insight_detail_endpoint = f"https://graph.microsoft.com/beta/copilot/users/{user_id}/onlineMeetings/{meeting_id}/aiInsights/{insight_id}"
                insight_detail_response = requests.get(insight_detail_endpoint, headers=headers)
                
                if insight_detail_response.status_code == 200:
                    insight_detail_data = insight_detail_response.json()
                    print(json.dumps(insight_detail_data, indent=2))
                    print()
                else:
                    print(f"Error fetching insight detail: {insight_detail_response.status_code}")
                    print(insight_detail_response.text)
                    print()
        else:
            print("No AI insights found for this meeting.")
    else:
        print(f"Error fetching meeting insights list: {insights_list_response.status_code}")
        print(insights_list_response.text)
else:
    print("No meeting ID available. Please run step 4 first.")